In [1]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import random
import math
import os
import pandas as pd

In [2]:
abstracts = pickle.load(open("pickles_data/abstracts_total.pkl", "rb"))

In [3]:
author_feat = pickle.load(open("pickles/author_level.pickle", "rb"))

In [4]:
author_feat['P14-1066']

[{'paper_name': 'scalable training of l1-regularized log-linear models.',
  'arnet_id': '53e99f2eb7602d97027ff009',
  'hindex': 40.5,
  'productivity': 371.5},
 {'paper_name': 'joint language and translation modeling with recurrent neural networks.',
  'arnet_id': '53e9afa6b7602d97039f69a4',
  'hindex': 11.5,
  'productivity': 42.75},
 {'paper_name': 'learning deep architectures for ai. fundamental trends',
  'arnet_id': '53e9afbab7602d9703a074c5',
  'hindex': 125.0,
  'productivity': 632.0},
 {'paper_name': 'a neural probabilistic language model.',
  'arnet_id': '53e9ab37b7602d97034c4455',
  'hindex': 40.75,
  'productivity': 177.5},
 {'paper_name': 'latent dirichlet allocation.',
  'arnet_id': '56d85479dabfae2eee1db078',
  'hindex': 0,
  'productivity': 0},
 {'paper_name': 'natural language processing (almost) from scratch.',
  'arnet_id': '53e99beab7602d970249335e',
  'hindex': 35.166666666666664,
  'productivity': 79.16666666666667},
 {'paper_name': 'indexing by latent semantic ana

In [5]:
location_feature = pickle.load(open("pickles_data/location_feature.pkl","rb"))
title_overlap = pickle.load(open("pickles_data/title_overlap.pkl","rb"))
context_count = pickle.load(open("pickles_data/context_count.pkl","rb"))
cue_words = pickle.load(open("pickles_data/cue_count.pkl", "rb"))
tags = pickle.load(open("pickles_data/baseline_tags.pkl", "rb"))
contexts = pickle.load(open("pickles_data/context_words.pkl", "rb"))
popularity = pickle.load(open("pickles_data/popularity_sent.pkl", "rb"))

In [6]:
paper_info = pickle.load(open("pickles/paper_info (1).pickle", "rb"))

In [7]:
# paper_info_1 = pickle.load(open("pickles/paper_info (1).pickle", "rb"))

In [8]:
abs_sim = pickle.load(open("pickles_data/abstract_similarity.pkl", "rb"))

In [9]:
paper_info[list(paper_info.keys())[0]]

[{'paper_name': 'scalable training of l1-regularized log-linear models.',
  'citation': 456,
  'abstract': 'The L-BFGS limited-memory quasi-Newton method is the algorithm of choice for optimizing the parameters of large-scale log-linear models with L2 regularization, but it cannot be used for an L1-regularized loss due to its non-differentiability whenever some parameter is zero. Efficient algorithms have been proposed for this task, but they are impractical when the number of parameters is very large. We present an algorithm Orthant-Wise Limited-memory Quasi-Newton (OWL-QN), based on L-BFGS, that can efficiently optimize the L1-regularized log-likelihood of log-linear models with millions of parameters. In our experiments on a parse reranking task, our algorithm was several orders of magnitude faster than an alternative algorithm, and substantially faster than L-BFGS on the analogous L2-regularized problem. We also present a proof that OWL-QN is guaranteed to converge to a globally op

In [10]:
year_diff = pickle.load(open("pickles_data/year_diff.pkl","rb"))

In [11]:
fixed_context = pickle.load(open("pickles_data/fixed_context.pkl","rb"))

In [12]:
weighted_cue = pickle.load(open("pickles_data/weighted_cue_words.pkl", "rb"))
bert_embeddings = pickle.load(open("pickles_data/bert_embeddings_relu.pkl", "rb"))

In [13]:
print(len(context_count.keys()))

1181


In [14]:
num_table = pickle.load(open("pickles_data/num_table.pkl","rb"))

In [15]:
cit_titles = pickle.load(open("pickles_data/citation_titles.pkl","rb"))
ids = tags.keys()
ids = list(set(ids).intersection(set(num_table.keys())))

In [16]:
print(len(ids))

1181


In [17]:
dataset = []
c = 0
cbert = 0
count_cc = 0
for key in ids :
    papers = cit_titles[key]
    for paper in papers :
        data = {}
        data['paper_id'] = key
        data['citation_title'] = paper
        
        if(key in bert_embeddings):
            embedding_found = 0
            for pap in bert_embeddings[key] :
                if(pap['paper_name']==paper) :
                    cbert+=1
                    data['bert_embed'] = pap['embedding']
                    embedding_found = 1
                    break
                    
            if(embedding_found==1) :
                
                if(key in paper_info) :
                    info_found = 0 
                    for pap in paper_info[key] :
                        if(pap['paper_name']==paper) :
                            try: 
                                data['citation_count'] = int(pap['citation'])
                            except :
                                data['citation_count'] = 1
                                count_cc += 1
                            try :
                                data['venue_cit'] = int(pap['venue_cit'])
                            except :
                                data['venue_cit'] = 20
                            
                            info_found = 1
                            break
                            
                    if(info_found==1) :
                        
                        for pap in abs_sim[key] :
                            if(pap['paper_name']==paper) :
                                data['abstract_sim'] = pap['abs_sim']
                                break
                                
                        for pap in location_feature[key]:
                            if(pap['paper_name']==paper) :
                                data['location_feature'] = pap['location_feature']
                                break
                                
                        for pap in author_feat[key] :
                            if(pap['paper_name']==paper) :
                                data['author_hindex'] = pap['hindex']
                                data['author_prod'] = pap['productivity']

                        for pap in num_table[key]:
                            if(pap['paper_name']==paper) :
                                data['num_table'] = pap['num_table']
                                break


                        for pap in year_diff[key]:
                            if(pap['paper_name']==paper) :
                                data['year_diff'] = pap['diff']
                                break

                        for pap in title_overlap[key]:
                            if(pap['paper_name']==paper) :
                                data['title_overlap'] = pap['overlap']
                                break

                        for pap in popularity[key]:
                            if(pap['paper_name']==paper) :
                                data['popularity'] = pap['popularity']
                                break

                        for pap in context_count[key]:
                            if(pap['paper_name']==paper) :
                                data['context_count'] = pap['context_count']
                                break

                        for pap in contexts[key] :
                            if(pap['paper_name']==paper) :
                                data['context'] = pap['context']
                                break

                        for pap in cue_words[key]:
                            if(pap['paper_name']==paper) :
                                data['cue_count'] = pap['cue_count']
                                break

                        for pap in weighted_cue[key]:
                            if(pap['paper_name']==paper) :
                                max_data = pap['cue_weights_max']
                                data['wcue_max'] = []
                                for key1 in max_data :
                                    data['wcue_max'].append(max_data[key1])
                                add_data = pap['cue_weights_add']
                                data['wcue_add'] = []
                                for key1 in add_data :
                                    data['wcue_add'].append(add_data[key1])
                                break

                        for pap in tags[key]:
                            if(pap['paper_name']==paper) :
                                if(pap['tag']==1):
                                    data['label'] = 'baseline'
                                else :
                                    data['label'] = 'non_baseline'
                                break

                        for pap in fixed_context[key]:
                            if(pap['paper_name']==paper) :
                                data['fixed_context'] = pap['fixed_context']                   
                        dataset.append(data)
                        
                        
        
print(c)
print(cbert)

0
27636


In [18]:
print(len(dataset))
print(count_cc)

26329
10


In [19]:
final_dataset = []
for data in dataset :
    if(data['context_count']!=0):
        final_dataset.append(data)
print(len(final_dataset))

26323


In [28]:
dataset = final_dataset

In [29]:
non_baselines = []
baselines = []
for data in dataset :
    if(data['label']=="baseline"):
        baselines.append(data)
    else :
        non_baselines.append(data)

In [30]:
print(len(baselines))
print(len(non_baselines))

2663
23660


In [31]:
ar = np.random.choice(len(non_baselines), len(baselines))

In [32]:
non_base = []
for x in ar :
    non_base.append(non_baselines[x])

In [33]:
baselines.extend(non_base)
dataset = baselines

In [34]:
print(len(dataset))

5326


In [35]:
random.shuffle(dataset)

In [36]:
trainset = dataset[:int(0.8*len(dataset))]
testset = dataset[int(0.8*len(dataset)):]

In [37]:
count_train = 0
count_test = 0
for data in trainset :
    if(data['label']=='baseline') :
        count_train+=1
for data in testset :
    if(data['label']=='baseline') :
        count_test+=1

print(count_train)
print(count_test)

2119
544


In [38]:
def sanity_check(probs):

	sum = 0
	for key, value in probs.items():
		sum+=value
	sum = round(sum, 7)
	return sum==1

In [39]:
def lm_model(trainset, testset):

    word_baseline_frequency = {}
    word_non_baseline_frequency = {}
    baseline_length = 0
    non_baseline_length = 0

    for i in trainset:
        # if len(i['fixed_context'])>1 and i['label']=='baseline':
        # 	continue
        for context in i['fixed_context']:
            if i['label']=='baseline':
                baseline_length+=len(context)
            else:
                non_baseline_length+=len(context)
            for word in context:
                if word not in word_baseline_frequency:
                    word_baseline_frequency[word] = 0
                    word_non_baseline_frequency[word] = 0
                if i['label']=='baseline':
                    word_baseline_frequency[word]+=1
                else:
                    word_non_baseline_frequency[word]+=1

    final_word_baseline_frequency = {}
    final_word_baseline_frequency['<unk>'] = 0
    word_non_baseline_frequency['<unk>'] = 0

    for word in word_baseline_frequency:
        if word_baseline_frequency[word]>5:	
            final_word_baseline_frequency[word] = word_baseline_frequency[word]
        else:
            final_word_baseline_frequency['<unk>'] += word_baseline_frequency[word]
            word_non_baseline_frequency['<unk>'] += word_non_baseline_frequency[word]
            del word_non_baseline_frequency[word]

    word_baseline_frequency = final_word_baseline_frequency

    word_baseline_probability = {}
    word_non_baseline_probability = {}

    for word in word_baseline_frequency:
        word_baseline_probability[word] = (word_baseline_frequency[word]+1)/(baseline_length+len(word_baseline_frequency))
        word_non_baseline_probability[word] = (word_non_baseline_frequency[word]+1)/(non_baseline_length+len(word_non_baseline_frequency))

    # print(word_baseline_probability['<unk>'])
    # print(word_non_baseline_probability['<unk>'])

    assert sanity_check(word_baseline_probability) and sanity_check(word_non_baseline_probability)

    conf_mat1 = np.zeros((2,2))
    conf_mat2 = np.zeros((2,2))

    min_p = 1

    for i in trainset+testset:
        P_b = 0
        P_nb = 0
        maxP_b = 0
        minP_nb = 1
        n_better = 0
        for context in i['fixed_context']:
            
            P_b_con = 1
            P_nb_con = 1
            for word in context:
                if word not in word_baseline_probability:
                    P_b_con*=word_baseline_probability['<unk>']
                    P_nb_con*=word_non_baseline_probability['<unk>']
                else:
                    P_b_con*=word_baseline_probability[word]
                    P_nb_con*=word_non_baseline_probability[word]
            if P_b_con>P_nb_con:
                n_better+=1
            P_b+=P_b_con
            P_nb+=P_nb_con
            maxP_b = max(P_b, P_b_con)
            minP_nb = min(P_nb, P_nb_con)
        P_b/=len(i['fixed_context'])
        P_nb/=len(i['fixed_context'])

        # if P_b>P_nb:
        # 	print('yes')

        # i['P_b'] = P_b
        # i['P_nb'] = P_nb
#         n_better = n_better/len(i['fixed_context'])
        min_p = min(min_p, P_b, P_nb)
#         i['lmp'] = [P_b, P_nb, n_better]
        i['lmp'] = [P_b, P_nb, maxP_b, minP_nb, P_b/P_nb, maxP_b/minP_nb, n_better]

    for i in trainset:	
        # if len(i['fixed_context'])>1 and i['label']=='baseline':
        # 	testset.append(i)
        # 	continue
        i['lmp'][0]/=min_p
        i['lmp'][1]/=min_p
        i['lmp'][2]/=min_p
        i['lmp'][3]/=min_p
        if i['lmp'][0]>i['lmp'][1]:
            if i['label']=='baseline':
                conf_mat1[0, 0]+=1
            else:
                conf_mat1[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat1[1, 0]+=1
            else:
                conf_mat1[1, 1]+=1
        if i['lmp'][2]>i['lmp'][3]:
            if i['label']=='baseline':
                conf_mat2[0, 0]+=1
            else:
                conf_mat2[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat2[1, 0]+=1
            else:
                conf_mat2[1, 1]+=1

    # print(conf_mat1)
    prec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
    rec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat1 = np.zeros((2,2))

    # print(conf_mat2)
    prec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
    rec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
#     conf_mat2 = np.zeros((2,2))

    for i in testset:
        i['lmp'][0]/=min_p
        i['lmp'][1]/=min_p
        i['lmp'][2]/=min_p
        i['lmp'][3]/=min_p
        if i['lmp'][0]>i['lmp'][1]:
            if i['label']=='baseline':
                conf_mat1[0, 0]+=1
            else:
                conf_mat1[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat1[1, 0]+=1
            else:
                conf_mat1[1, 1]+=1
        if i['lmp'][2]>i['lmp'][3]:
            if i['label']=='baseline':
                conf_mat2[0, 0]+=1
            else:
                conf_mat2[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat2[1, 0]+=1
            else:
                conf_mat2[1, 1]+=1

    # print(conf_mat1)
    prec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
    rec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat1 = np.zeros((2,2))

    # print(conf_mat2)
    prec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
    rec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat2 = np.zeros((2,2))

In [40]:
lm_model(trainset,testset)

0.8105153565851119 0.7347805568664464 0.7707920792079207
0.7762882279011312 0.8744690891930156 0.8224589436307147
0.7521367521367521 0.6470588235294118 0.6956521739130436
0.7686143572621035 0.8644386030792339 0.8137150936726759


In [ ]:
# def context_feature(trainset, testset) :
#     total_counts = {}
#     citation_frequency = {}
#     paper_frequency = {}
#     curr_doc = trainset[0]['paper_id']
#     paper_words = set([])

#     for i in trainset:
#         if i['paper_id']!=curr_doc:
#             for word in paper_words:
#                 if word not in paper_frequency:
#                     paper_frequency[word] = 0
#                 paper_frequency[word]+=1
#             paper_words = set([])
#             curr_doc = i['paper_id']
#         words = set(i['context'])
#         paper_words = paper_words.union(words)
#         for word in words:
#             if word not in total_counts:
#                 total_counts[word] = 0
#                 citation_frequency[word] = 0
#             citation_frequency[word]+=1
#         counts = {word:0 for word in words}
#         for word in i['context']:
#             counts[word]+=1
#             total_counts[word]+=1

#     for word in paper_words:
#         if word not in paper_frequency:
#             paper_frequency[word] = 0
#         paper_frequency[word]+=1

#     final_citation_frequency = {word:count for word, count in citation_frequency.items()}
#     for i in citation_frequency:
#         if paper_frequency[i]<=20:
#             del total_counts[i]
#             del final_citation_frequency[i]

#     citation_frequency = final_citation_frequency
    
#     dataset = trainset+testset
    
#     idfs = {}

#     for word in citation_frequency:
#         idfs[word] = math.log10(len(dataset)*0.8/citation_frequency[word])
    
#     total_counts['<unk>'] = 0
    
#     for i in range(len(dataset)):
#         words = set(dataset[i]['context']) & set(total_counts.keys())
#         unigram_counts = {word:0 for word in words}
#         try:	
#             del unigram_counts['<SOS>']
#             del unigram_counts['<EOS>']
#         except:
#             pass

#         for j in range(len(dataset[i]['context'])):
#             word = dataset[i]['context'][j]
#             if word not in unigram_counts:
#                 continue
#             else:
#                 unigram_counts[word]+=1-dataset[i]['distances'][j]


#         dataset[i]['unigrams'] = unigram_counts


#     total_unigram_counts = total_counts
#     features = list(total_unigram_counts.keys())
#     ngram_to_idx = {features[i]:i for i in range(len(features))}
    
#     pickle.dump(total_counts, open('pickles/total_counts.pkl', 'wb'))
#     pickle.dump(dataset, open('pickles/data_ngram.pkl', 'wb'))
#     pickle.dump(ngram_to_idx, open('pickles/ngram_to_idx.pkl', 'wb'))
#     pickle.dump(idfs, open('pickles/idfs.pkl', 'wb'))
    
#     data_mat = np.zeros((len(dataset), len(ngram_to_idx)+35+1+5+1))

#     for i in range(len(dataset)):
#         context_counts = {}
#         for word in dataset[i]['unigrams']:
#             if word not in context_counts:
#                 context_counts[word] = 0
#             context_counts[word]+=dataset[i]['unigrams'][word]
#             if word in idfs:
#                 data_mat[i, ngram_to_idx[word]] = dataset[i]['unigrams'][word]

#         dataset[i]['context_feature'] = data_mat[i, :len(ngram_to_idx)]
        
#     return dataset

In [ ]:
# dataset = context_feature(trainset, testset)

In [41]:
dataset = trainset+testset

In [ ]:
# for data in dataset :
#     wcue_max = data['wcue_max']
#     cue_list = ['among', 'precis', 'implement', 'maximum', 'experi', 'strategi', 
#  'overal', 'previou', 'correl', 'calcul', 'achiev', 'gold', 
#  'top', 'obtain', 'significantli', 'report', 'best', 'evalu', 
#  'result', 'base', 'modifi', 'extend', 'metric', 'baselin', 
#  'accord', 'perform', 'recal', 'standard', 'figur', 'comparison',
#  'yield', 'outperform', 'increas', 'stateoftheart', 'accuraci', 'method', 
#  'procedur', 'score', 'origin', 'highest', 'signific', 'higher', 
#  'fscore', 'compar']
#     final_ar = []
#     rem_word = ['method', 'recal', 'correl', 'figur', 'origin', 'among', 'precis', 'fscore', 'procedur', 'score']
#     for i in range(len(wcue_max)):
#         if(cue_list[i] not in rem_word):
#             final_ar.append(wcue_max[i])
#     data['wcue_max'] = final_ar

In [42]:
for data in dataset :
    print(len(data['wcue_max']))
    break

44


In [43]:
# 1+1+1+1+ 1+1+1+ 6+7+ 44+256
values = []
output = []
for data in dataset :
    ar = []
    count = 0
    ar.append(data['context_count'])   
    ar.append(data['title_overlap'])
#     ar.append(data['cue_count'])
#     ar.append(sum(data['location_feature']))
#     ar.append(sum(data['wcue_add']))
#     ar.append(data['abstract_sim'])
    ar.append(data['author_hindex'])
    ar.append(data['author_prod'])
#     ar.append(data['venue_cit'])
    ar.append(data['popularity'])
    ar.append(data['citation_count'])
    ar.append(data['num_table'])
    ar.extend(data['location_feature'])
    ar.extend(data['lmp'])
    ar.extend(data['wcue_max'])
    ar.extend(data['bert_embed'])
    values.append(ar)
    if(data['label']=='baseline'):
        output.append(1)
    else :
        output.append(0)

In [44]:
values = np.array(values)
output = np.array(output)

In [45]:
df = pd.DataFrame(values)

In [46]:
print(values.shape)

(5326, 320)


In [47]:
# std_scale = MinMaxScaler((0,1))
# values[:,0:-6] = std_scale.fit_transform(values[:,0:-6])
scaler = MinMaxScaler((0,1))
values = scaler.fit_transform(values)

In [48]:
def split(data, output):
    n = len(data)
    last = int(0.8*n)
    train_data = data[:last]
    train_output = output[:last]
    test_data = data[last:]
    test_output = output[last:]
    return train_data, test_data, train_output, test_output

In [49]:
train_data, test_data, train_output, test_output = split(values, output)

In [50]:
def shuffle(train_data, train_output) :
    baselines = []
    non_baselines = []
    for i in range(len(train_output)) :
        if(train_output[i]==1) :
            baselines.append(train_data[i])
        else :
            non_baselines.append(train_data[i])
    
    n = len(baselines)
    ar = np.random.choice(len(non_baselines), len(baselines))
    nb_ar = []
    for x in ar :
        nb_ar.append(non_baselines[x])
        
    data = []
    data.extend(nb_ar)
    output = []
    for i in range(len(data)):
        output.append(0)
    
    data.extend(baselines)
    for i in range(n) :
        output.append(1)
        
    total_data = []
    for i in range(len(data)):
        ar = []
        ar.append(data[i])
        ar.append(output[i])
        total_data.append(ar)
        
    total_data = np.array(total_data)
    np.random.shuffle(total_data)
    
    data = []
    output = []
    for ar in total_data :
        data.append(ar[0])
        output.append(ar[1])
        
    data = np.array(data)
    output = np.array(output)
    
    return data, output

In [ ]:
# train_data, train_output = shuffle(train_data, train_output)

In [53]:
params = {'C':[0.1, 0.5, 1, 2], 'fit_intercept':[True, False]}
modelin = LogisticRegression(solver='lbfgs', max_iter=1e4, n_jobs=5, random_state=1, warm_start=False)
model = GridSearchCV(modelin, params, cv=2, n_jobs=5)
# model = LogisticRegression(solver='lbfgs',max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, C=1, fit_intercept=True)
clf = model.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))
predictions = clf.predict(train_data)
print(classification_report(train_output, predictions))
print(model.best_params_)
# print(type(model.best_estimator_))

import copy
normal = copy.deepcopy(clf.best_estimator_.coef_[0])
coef = clf.best_estimator_.coef_[0][:64]
print(coef)
ar = clf.best_estimator_.coef_[0]
ar.sort()
# print(ar)

for i in range(10) :
    print(ar[len(ar)-1-i], list(normal).index(ar[len(ar)-1-i]))
# print(clf.best_estimator_.coef_[0])

              precision    recall  f1-score   support

           0       0.84      0.90      0.87       522
           1       0.89      0.84      0.87       544

    accuracy                           0.87      1066
   macro avg       0.87      0.87      0.87      1066
weighted avg       0.87      0.87      0.87      1066

              precision    recall  f1-score   support

           0       0.90      0.93      0.92      2141
           1       0.93      0.90      0.91      2119

    accuracy                           0.91      4260
   macro avg       0.91      0.91      0.91      4260
weighted avg       0.91      0.91      0.91      4260

{'C': 2, 'fit_intercept': False}
[ 1.95e+00  1.64e+00 -1.02e+00  8.69e-01  7.12e-02 -5.39e-01  6.00e+00
  6.34e-01  6.55e-01  6.55e+00 -1.00e-01 -1.20e+00 -2.10e+00 -7.69e-02
 -1.13e-01 -7.69e-02 -1.13e-01  3.24e-01  2.51e-03  1.23e+01  8.26e-01
 -1.58e-01  8.69e-01  1.70e+00 -9.42e-01 -2.32e-01  8.48e-02  1.44e+00
 -2.95e-01  5.84e-01  6.06e-0

In [52]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
params = {'C':[0.1, 0.5, 1, 2, 3], 'class_weight':[{0:1,1:2}, {0:1,1:3}, {0:1,1:4}, {0:1,1:5}]}
rbf_svc = svm.SVC(kernel='rbf', gamma='scale', max_iter=1e4, C=1 )
# model = GridSearchCV(rbf_svc, params, cv=2, n_jobs=5)
clf = rbf_svc.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
class_names = ['non_baseline','baseline']
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, test_data, test_output,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       522
           1       0.89      0.85      0.87       544

    accuracy                           0.87      1066
   macro avg       0.87      0.87      0.87      1066
weighted avg       0.87      0.87      0.87      1066

Confusion matrix, without normalization
[[467  55]
 [ 80 464]]
Normalized confusion matrix
[[0.89 0.11]
 [0.15 0.85]]


<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

In [54]:
params = {'criterion':['entropy', 'gini'], 'splitter':['best', 'random'], 'min_samples_split':[2, 10, 20, 40],  'min_samples_leaf':[2, 10, 20, 40], 'max_depth':[20, 50, 100]}
model = DecisionTreeClassifier(random_state=1, presort=True, criterion='entropy', max_depth=20,min_samples_leaf=2,min_samples_split=2,splitter='best')
# clf = GridSearchCV(model, params, cv=2, n_jobs=5)
clf = model.fit(train_data, train_output)
predictions = clf.predict(test_data)
print(classification_report(test_output, predictions))

/home/tanisha17116/.local/lib/python3.6/site-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.80      0.83      0.81       522
           1       0.83      0.80      0.81       544

    accuracy                           0.81      1066
   macro avg       0.81      0.81      0.81      1066
weighted avg       0.81      0.81      0.81      1066



In [ ]:
print(clf.best_params_)

In [55]:
model = RandomForestClassifier(random_state=1, n_jobs=5, max_leaf_nodes=200, class_weight={0:1,1:5})
clf = model.fit(train_data, train_output)
predictions = clf.predict(test_data)
print(classification_report(test_output, predictions))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87       522
           1       0.87      0.87      0.87       544

    accuracy                           0.87      1066
   macro avg       0.87      0.87      0.87      1066
weighted avg       0.87      0.87      0.87      1066



In [ ]:
model = KNeighborsClassifier(n_jobs=5, n_neighbors=5)
clf = model.fit(train_data, train_output)
predictions = clf.predict(test_data)
print(classification_report(test_output, predictions))

In [ ]:
model = GaussianNB()
clf = model.fit(train_data, train_output)
predictions = clf.predict(test_data)
print(classification_report(test_output, predictions))

In [ ]:
params = {'hidden_layer_sizes':[(500, 100), (50,)], 'alpha':[10, 0.1, 0.001, 0.00001]}
model = MLPClassifier(random_state=1, learning_rate='constant', batch_size=1500, max_iter=200, warm_start=True, hidden_layer_sizes=(500,100), alpha=0.01)
#clf = GridSearchCV(model, params, cv=2, n_jobs=5)
clf = model.fit(train_data, train_output)
predictions = clf.predict(test_data)
print(classification_report(test_output, predictions))
#print(clf.best_params_)

In [ ]:
coef = model.best_estimator_.coef_

In [ ]:
# method, recal, correl, figur, origin, among, precis, fscore, procedur, score, origin

In [ ]:
coef1 = coef[0][0:4]
['among', 'precis', 'implement', 'maximum', 'experi', 'strategi', 
 'overal', 'previou', 'correl', 'calcul', 'achiev', 'gold', 
 'top', 'obtain', 'significantli', 'report', 'best', 'evalu', 
 'result', 'base', 'modifi', 'extend', 'metric', 'baselin', 
 'accord', 'perform', 'recal', 'standard', 'figur', 'comparison',
 'yield', 'outperform', 'increas', 'stateoftheart', 'accuraci', 'method', 
 'procedur', 'score', 'origin', 'highest', 'signific', 'higher', 
 'fscore', 'compar']
print(coef1)

In [ ]:
for data in dataset :
    print(len(data['wcue_max']))
    break

In [ ]:
base_as_nb = []
nb_as_base = []
base_as_base = []
nb_as_nb = []

for i in range(len(predict_test)):
    testset[i].pop('fixed_context')
    testset[i].pop('context')
    testset[i].pop('bert_embed')
    if(test_output[i]==1 and predict_test[i]==0) :
        base_as_nb.append(testset[i])
    if(test_output[i]==0 and predict_test[i]==1) :
        nb_as_base.append(testset[i])
    if(test_output[i]==1 and predict_test[i]==1) :
        base_as_base.append(testset[i])
    if(test_output[i]==0 and predict_test[i]==0) :
        nb_as_nb.append(testset[i])
        

In [ ]:
pickle.dump(nb_as_base, open("nb_as_base.pkl","wb"))
pickle.dump(base_as_nb, open("base_as_nb.pkl","wb"))